<a href="https://colab.research.google.com/github/yusufakcakaya/Algorythm-NLP-Entity-Recognition/blob/Nichelle/Relation_extraction_OpenNRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source: 1. https://analyticsindiamag.com/beginners-guide-to-opennre-toolkit/ 
        2. https://towardsdatascience.com/from-text-to-knowledge-the-information-extraction-pipeline-b65e7e30273e


## Connecting to google drive

In [95]:
## Connecting google drive

In [4]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


## Cloning OpenNRE 

In [5]:
!git clone https://github.com/thunlp/OpenNRE.git

Cloning into 'OpenNRE'...
remote: Enumerating objects: 1514, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 1514 (delta 81), reused 82 (delta 42), pack-reused 1377
Receiving objects: 100% (1514/1514), 266.83 MiB | 26.54 MiB/s, done.
Resolving deltas: 100% (901/901), done.


In [6]:
cd OpenNRE

/content/OpenNRE


In [7]:
!pip install -r requirements.txt

     |████████████████████████████████| 748.8 MB 17 kB/s 
     |████████████████████████████████| 1.3 MB 42.9 MB/s 
     |████████████████████████████████| 234 kB 42.4 MB/s 
     |████████████████████████████████| 7.0 MB 43.7 MB/s 
     |████████████████████████████████| 1.5 MB 46.6 MB/s 
     |████████████████████████████████| 1.2 MB 48.8 MB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
     |████████████████████████████████| 2.9 MB 26.9 MB/s 
     |████████████████████████████████| 748 kB 44.8 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+

## Loading pretrained model from opennre

In [8]:
import opennre

In [25]:
model = opennre.get_model('wiki80_cnn_softmax')

2022-01-20 13:30:32,023 - root - INFO - Initializing word embedding with word2vec.


In [10]:
model.infer({'text': 'He was the son of ashok, and grandson of rahul.', 'h': {'pos': (18, 46)}, 't': {'pos': (78, 91)}})

('child', 0.3211628794670105)

## Entity linking

In [69]:
article = "Elon Musk is a business magnate, industrial designer, and engineer. Elon Musk is the founder, CEO, CTO, and chief designer of SpaceX. Elon Musk is also early investor, CEO, and product architect of Tesla, Inc. Elon Musk is also the founder of The Boring Company and the co-founder of Neuralink. A centibillionaire, Musk became the richest person in the world in January 2021, with an estimated net worth of $185 billion at the time, surpassing Jeff Bezos. Musk was born to a Canadian mother and South African father and raised in Pretoria, South Africa. Elon Musk briefly attended the University of Pretoria before moving to Canada aged 17 to attend Queen's University. Elon Musk transferred to the University of Pennsylvania two years later, where Elon Musk received dual bachelor's degrees in economics and physics. Elon Musk moved to California in 1995 to attend Stanford University, but decided instead to pursue a business career. Elon Musk went on co-founding a web software company Zip2 with Elon Musk brother Kimbal Musk."

In [96]:
article1 = "Residents of Brussels with solar panels installed can expect to receive around 10% more green certificates for the electricity generated in 2022 than they did last year. The Region wants to continue to guarantee the repayment of investments in solar over seven years, Bruzz reports, following advice to this effect from energy regulator Brugel. In 2021, the number of green certificates (GSCs) for new solar panels in Brussels decreased by 20 to 46%, depending on the power category of the installation, to mirror the falling cost of installations over the last several years (in Flanders and Wallonia, green certificates were abolished back in 2014). But now the subsidy will rise by around 10% for each category, as the cost of installing solar panels begins to climb again. “Energy regulator Brugel noticed that the cost price of the installations has increased due to the rising price of raw materials,” said Raphaël Boucquey, legal advisor for renewable energy at Brussels Environment. “The aim is to be able to continue to guarantee a return on investment in solar panels within seven years thanks to the 10% increase in green certificates.” The increase in the number of green certificates only applies to new installations from 1 January 2022, and is not retroactive."

In [97]:
import urllib
from string import punctuation
import nltk

ENTITY_TYPES = ["human", "person", "company", "enterprise", "business", "geographic region",
                "country", "human settlement", "geographic entity", "territorial entity type", "organization"]

def wikifier(text, lang="en", threshold=0.8):
    """Function that fetches entity linking results from wikifier.com API"""
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "tgbdmkpmkluegqfbawcwjywieevmza"),
        ("pageRankSqThreshold", "%g" %
         threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "100"), ("nWordsToIgnoreFromList", "100"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "2"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
    ])
    url = "http://www.wikifier.org/annotate-article"
    # Call the Wikifier and read the response.
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout=60) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    results = list()
    for annotation in response["annotations"]:
        # Filter out desired entity classes
        if ('wikiDataClasses' in annotation) and (any([el['enLabel'] in ENTITY_TYPES for el in annotation['wikiDataClasses']])):

            # Specify entity label
            if any([el['enLabel'] in ["human", "person"] for el in annotation['wikiDataClasses']]):
                label = 'Person'
            elif any([el['enLabel'] in ["country", "geographic region", "human settlement", "geographic entity", "territorial entity type"] for el in annotation['wikiDataClasses']]):
                label = 'Location'
            elif any([el['enLabel'] in ["company", "enterprise", "business", "organization"] for el in annotation['wikiDataClasses']]):
                label = 'Organization'
    
            else:
                label = None

            results.append({'title': annotation['title'], 'wikiId': annotation['wikiDataItemId'], 'label': label,
                            'characters': [(el['chFrom'], el['chTo']) for el in annotation['support']]})
    return results

In [98]:
import json

In [99]:
linking = wikifier(article1)

In [100]:
linking

[{'characters': [(13, 20), (418, 425), (969, 976)],
  'label': 'Location',
  'title': 'Brussels',
  'wikiId': 'Q240'},
 {'characters': [(580, 587)],
  'label': 'Location',
  'title': 'Flanders',
  'wikiId': 'Q234'},
 {'characters': [(593, 600)],
  'label': 'Location',
  'title': 'Wallonia',
  'wikiId': 'Q231'}]

## Entity relation extraction

In [101]:
import itertools 

In [102]:
import opennre

In [103]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [104]:
def ie_pipeline(text, relation_threshold=0.9, entities_threshold=0.8):
    # Prepare the URL.
    data = urllib.parse.urlencode([
        ("text", text), ("relation_threshold", relation_threshold),
        ("entities_threshold", entities_threshold)])
    
    url = "http://localhost:5000?" + data
    req = urllib.request.Request(url, data=data.encode("utf8"), method="GET")
    with urllib.request.urlopen(req, timeout=150) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
    # Output the annotations.
    return response

In [105]:
def strip_punctuation(s):
    """Removes all punctuation from a string"""
    return ''.join(c for c in s if c not in punctuation)

In [106]:
def deduplicate_dict(d):
    return [dict(y) for y in set(tuple(x.items()) for x in d)]

In [111]:
def hello_ie():

  relations_list = list()
  entities_list = list()
  text = article
  for sentence in nltk.sent_tokenize(text):
      sentence = strip_punctuation(sentence)
      entities = wikifier(sentence, threshold=0.8)
      entities_list.extend(
          [{'title': el['title'], 'wikiId': el['wikiId'], 'label': el['label']} for el in entities])
      # Iterate over every permutation pair of entities
      for permutation in itertools.permutations(entities, 2):
          for source in permutation[0]['characters']:
              for target in permutation[1]['characters']:
                  # Relationship extraction with OpenNRE
                  data = model.infer(
                      {'text': sentence, 'h': {'pos': [source[0], source[1] + 1]}, 't': {'pos': [target[0], target[1] + 1]}})
                  if data[1] > 0.9:
                      relations_list.append(
                          {'source': permutation[0]['title'], 'target': permutation[1]['title'], 'type': data[0]})

  return {'entities': deduplicate_dict(entities_list), 'relations': deduplicate_dict(relations_list)}



In [112]:
w = hello_ie()
w

{'entities': [{'label': 'Location',
   'title': 'University of Pretoria',
   'wikiId': 'Q604444'},
  {'label': 'Organization', 'title': 'SpaceX', 'wikiId': 'Q193701'},
  {'label': 'Organization', 'title': 'Neuralink', 'wikiId': 'Q29043471'},
  {'label': 'Organization',
   'title': 'The Boring Company',
   'wikiId': 'Q28874479'},
  {'label': 'Location', 'title': 'Pretoria', 'wikiId': 'Q3926'},
  {'label': 'Location',
   'title': 'University of Pennsylvania',
   'wikiId': 'Q49117'},
  {'label': 'Person', 'title': 'Jeff Bezos', 'wikiId': 'Q312556'},
  {'label': 'Organization', 'title': 'Tesla, Inc.', 'wikiId': 'Q478214'},
  {'label': 'Person', 'title': 'Elon Musk', 'wikiId': 'Q317521'},
  {'label': 'Person', 'title': 'Kimbal Musk', 'wikiId': 'Q6409751'},
  {'label': 'Location', 'title': 'Stanford University', 'wikiId': 'Q41506'}],
 'relations': [{'source': 'University of Pennsylvania',
   'target': 'Elon Musk',
   'type': 'subsidiary'}]}

In [82]:
import pandas as pd

In [94]:
df = pd.DataFrame(list(w.items()), columns=['column1','column2']).T

In [92]:
df

,0,1
0,entities,relations
1,"[{'title': 'University of Pretoria', 'wikiId':...","[{'source': 'University of Pennsylvania', 'tar..."
